conda create -n drowning_detection_Enhanced python=3.10
conda activate drowning_detection_Enhanced
pip install -r drowning_detection.txt
pip install opencv-python tensorflow


In [4]:
import sys
import time
import cv2
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, 
                             QPushButton, QFileDialog, QHBoxLayout, QGroupBox, QSlider)
from PyQt5.QtGui import QImage, QPixmap, QFont, QPalette, QColor
from PyQt5.QtCore import Qt, QThread, pyqtSignal
from ultralytics import YOLO
import torch

In [5]:
class VideoThread(QThread):
    change_pixmap = pyqtSignal(QImage, list, int, bool)
    alert_signal = pyqtSignal(bool)

    def __init__(self, source=None):
        super().__init__()
        self.source = source
        self.running = True
        self.model = YOLO('yolov9c.pt')
        self.drowning_threshold = 0.7
        self.proximity_threshold = 50
        self.track_history = {}
        self.alert_status = False
        self.alert_counter = 0
        self.alert_cooldown = 30  # Frames to maintain alert after detection

    def run(self):
        cap = cv2.VideoCapture(self.source if self.source else 0)
        if not cap.isOpened():
            print("Error opening video source")
            return

        while self.running:
            ret, frame = cap.read()
            if not ret:
                break

            # Detect persons with YOLOv9
            results = self.model.track(
                frame, 
                persist=True, 
                classes=[0],  # Only detect persons
                conf=0.5,
                tracker="botsort.yaml"
            )
            
            annotated_frame = results[0].plot()
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.int().cpu().numpy() if results[0].boxes.id is not None else []
            confidences = results[0].boxes.conf.cpu().numpy()
            
            person_count = len(boxes)
            centroids = []
            person_info = []
            drowning_flags = []
            
            # Process each detected person
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box[:4])
                person_img = frame[y1:y2, x1:x2]
                
                if person_img.size == 0:
                    continue
                    
                # Drowning detection logic (simplified - replace with your model)
                is_drowning = False
                if person_count == 1 and confidences[i] > 0.7:
                    is_drowning = True
                
                # Track centroids for proximity analysis
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                centroids.append(centroid)
                drowning_flags.append(is_drowning)
                
                # Store person info for GUI display
                person_info.append({
                    "id": track_ids[i] if i < len(track_ids) else i,
                    "position": (x1, y1, x2, y2),
                    "drowning": is_drowning
                })
            
            # Proximity analysis for multi-person scenarios
            for i in range(len(centroids)):
                for j in range(i+1, len(centroids)):
                    dist = np.sqrt((centroids[i][0]-centroids[j][0])**2 + 
                                  (centroids[i][1]-centroids[j][1])**2)
                    if dist < self.proximity_threshold and (drowning_flags[i] or drowning_flags[j]):
                        self.alert_status = True
                        self.alert_counter = self.alert_cooldown
            
            # Handle alert status
            if any(drowning_flags):
                self.alert_status = True
                self.alert_counter = self.alert_cooldown
            
            # Decrement alert counter
            if self.alert_counter > 0:
                self.alert_counter -= 1
            else:
                self.alert_status = False
            
            # Convert to RGB for display
            rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
            
            # Emit signals to update GUI
            self.change_pixmap.emit(qt_image, person_info, person_count, self.alert_status)
            self.alert_signal.emit(self.alert_status)
            
            # Control frame rate
            time.sleep(0.03)
        
        cap.release()

    def stop(self):
        self.running = False
        self.wait()

In [6]:
class DrowningDetectionApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("AI Drowning Detection System")
        self.setGeometry(100, 100, 1200, 800)
        
        # Central widget and layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)
        
        # Video display
        self.video_label = QLabel(self)
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setMinimumSize(800, 600)
        self.video_label.setStyleSheet("background-color: black;")
        
        # Control panel
        control_group = QGroupBox("Controls")
        control_layout = QVBoxLayout()
        
        # Alert panel
        self.alert_panel = QLabel("NO ALERTS")
        self.alert_panel.setAlignment(Qt.AlignCenter)
        self.alert_panel.setFont(QFont("Arial", 24, QFont.Bold))
        self.alert_panel.setStyleSheet("background-color: green; color: white;")
        self.alert_panel.setMinimumHeight(80)
        
        # Statistics panel
        self.stats_label = QLabel("Persons Detected: 0")
        self.stats_label.setFont(QFont("Arial", 14))
        self.stats_label.setAlignment(Qt.AlignCenter)
        
        # Person info panel
        self.person_info_label = QLabel("No persons detected")
        self.person_info_label.setFont(QFont("Arial", 12))
        self.person_info_label.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        self.person_info_label.setWordWrap(True)
        
        # Buttons
        self.btn_open = QPushButton("Open Video File")
        self.btn_open.clicked.connect(self.open_file)
        
        self.btn_camera = QPushButton("Start Camera")
        self.btn_camera.clicked.connect(self.start_camera)
        
        self.btn_stop = QPushButton("Stop Detection")
        self.btn_stop.clicked.connect(self.stop_detection)
        
        # Threshold controls
        prox_slider_layout = QHBoxLayout()
        prox_label = QLabel("Proximity Threshold:")
        prox_label.setFont(QFont("Arial", 10))
        self.prox_slider = QSlider(Qt.Horizontal)
        self.prox_slider.setRange(20, 200)
        self.prox_slider.setValue(50)
        self.prox_slider.valueChanged.connect(self.update_proximity_threshold)
        
        # Add widgets to layouts
        control_layout.addWidget(self.alert_panel)
        control_layout.addWidget(self.stats_label)
        control_layout.addWidget(self.person_info_label)
        control_layout.addStretch(1)
        
        prox_slider_layout.addWidget(prox_label)
        prox_slider_layout.addWidget(self.prox_slider)
        control_layout.addLayout(prox_slider_layout)
        
        control_layout.addWidget(self.btn_open)
        control_layout.addWidget(self.btn_camera)
        control_layout.addWidget(self.btn_stop)
        control_group.setLayout(control_layout)
        
        # Main layout
        main_layout.addWidget(self.video_label, 70)
        main_layout.addWidget(control_group, 30)
        
        # Video thread
        self.video_thread = None
        
        # Alert styling
        self.normal_alert_style = "background-color: green; color: white;"
        self.warning_alert_style = "background-color: red; color: white;"
        
    def open_file(self):
        file_name, _ = QFileDialog.getOpenFileName(
            self, "Open Video File", "", "Video Files (*.mp4 *.avi *.mov)"
        )
        if file_name:
            self.stop_detection()
            self.start_detection(file_name)
    
    def start_camera(self):
        self.stop_detection()
        self.start_detection(0)  # 0 for default camera
        
    def start_detection(self, source):
        self.video_thread = VideoThread(source)
        self.video_thread.change_pixmap.connect(self.update_image)
        self.video_thread.alert_signal.connect(self.update_alert_status)
        self.video_thread.start()
        
    def stop_detection(self):
        if self.video_thread and self.video_thread.isRunning():
            self.video_thread.stop()
            self.video_thread = None
            
    def update_image(self, image, person_info, person_count, alert_status):
        # Update video display
        pixmap = QPixmap.fromImage(image)
        self.video_label.setPixmap(pixmap.scaled(
            self.video_label.width(), 
            self.video_label.height(),
            Qt.KeepAspectRatio,
            Qt.SmoothTransformation
        ))
        
        # Update statistics
        self.stats_label.setText(f"Persons Detected: {person_count}")
        
        # Update person info
        info_text = ""
        for idx, person in enumerate(person_info):
            status = "DROWNING!" if person["drowning"] else "Normal"
            color = "red" if person["drowning"] else "green"
            info_text += (
                f"<b>Person {person['id']}:</b> "
                f"<span style='color:{color};'>{status}</span><br>"
                f"Position: {person['position']}<br><br>"
            )
        self.person_info_label.setText(info_text or "No persons detected")
        
        # Update alert status
        self.update_alert_status(alert_status)
    
    def update_alert_status(self, alert_status):
        if alert_status:
            self.alert_panel.setText("DROWNING ALERT!")
            self.alert_panel.setStyleSheet(self.warning_alert_style)
        else:
            self.alert_panel.setText("NO ALERTS")
            self.alert_panel.setStyleSheet(self.normal_alert_style)
            
    def update_proximity_threshold(self, value):
        if self.video_thread:
            self.video_thread.proximity_threshold = value
            
    def closeEvent(self, event):
        self.stop_detection()
        event.accept()

In [7]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = DrowningDetectionApp()
    window.show()
    sys.exit(app.exec_())


0: 640x384 (no detections), 326.9ms
Speed: 4.0ms preprocess, 326.9ms inference, 941.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 285.0ms
Speed: 1.0ms preprocess, 285.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 306.7ms
Speed: 1.5ms preprocess, 306.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 304.3ms
Speed: 1.0ms preprocess, 304.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 306.1ms
Speed: 2.0ms preprocess, 306.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 298.3ms
Speed: 1.0ms preprocess, 298.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 310.0ms
Speed: 1.0ms preprocess, 310.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 299.9ms
Speed: 1.0ms pre

SystemExit: 0

c:\Users\husse\.conda\envs\drowning_detection_Enhanced\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
